<a href="https://colab.research.google.com/github/ilyakozhevnikov/MyProjects/blob/main/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way.

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [1]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [98]:
train_df['text'][0]

'При этом, продолжает Кудрин, лично он уверен: нацпроект, например, обязан быть целостным, то есть работы по нему должны вестись без "размывания" с другими задачами или целями.'

In [85]:
import pandas as pd


train_df = pd.read_csv('df_train.csv').reset_index().rename(columns={'index': 'ID'})
train_df = pd.concat(
    [
        train_df[['ID', 'text']],
        pd.get_dummies(train_df['class'])
    ],
    axis=1
)

valid_df = train_df.sample(5_000, random_state=42)
train_df = train_df[~train_df.ID.isin(valid_df.ID)]
test_df = train_df.sample(5_000, random_state=42)
train_df = train_df[~train_df.ID.isin(test_df.ID)]

train_df = train_df.to_dict()
valid_df = valid_df.to_dict()
test_df = test_df.to_dict()

In [86]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_dict(train_df)
valid_dataset = Dataset.from_dict(valid_df)
test_dataset = Dataset.from_dict(test_df)

dataset = DatasetDict(
    {
        'train': train_dataset,
        'validation': valid_dataset,
        'test': test_dataset
    }
)

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [87]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'O', 'Авиакатастрофа', 'Акционерный конфликт', 'Беспорядки, конфликты и войны', 'Взрыв газа', 'Взятки', 'Возбуждение уголовного дела', 'Выемки', 'Жалобы потребителей', 'Заключение под стражу', 'Закрытие предприятия или производства', 'Закрытие филиала, представительства', 'Землетрясение', 'Извержение вулкана', 'Катастрофы, крушения', 'Кораблекрушение', 'Коррупция', 'Манипулирование рынком', 'Мошенничество', 'Наводнение', 'Нападение', 'Нарушение закона', 'Нарушение обязанностей', 'Нарушение прав акционеров', 'Обвал', 'Отзыв продукции', 'Отмывание денег', 'Падение метеорита', 'Падение прибыли', 'Падение продаж', 'Поддержка террористов', 'Пожары', 'Привлечение к уголовной ответственности', 'Проблемы с активами', 'Противодействие коррупции', 'Растрата денежных средств', 'Рост убытков', 'Стихийные бедствия', 'Теракт', 'Техногенные катастрофы', 'Убийство', 'Убытки', 'Уголовное задержание', 'Уголовное обвинение', 'Утечка нефт

Let's check the first example of the training split:

In [88]:
example = dataset['train'][0]
example

{'ID': 0,
 'text': 0,
 'O': 0,
 'Авиакатастрофа': 0,
 'Акционерный конфликт': 0,
 'Беспорядки, конфликты и войны': 0,
 'Взрыв газа': 0,
 'Взятки': 0,
 'Возбуждение уголовного дела': 0,
 'Выемки': 0,
 'Жалобы потребителей': 0,
 'Заключение под стражу': 0,
 'Закрытие предприятия или производства': 0,
 'Закрытие филиала, представительства': 0,
 'Землетрясение': 0,
 'Извержение вулкана': 0,
 'Катастрофы, крушения': 0,
 'Кораблекрушение': 0,
 'Коррупция': 0,
 'Манипулирование рынком': 0,
 'Мошенничество': 0,
 'Наводнение': 0,
 'Нападение': 0,
 'Нарушение закона': 0,
 'Нарушение обязанностей': 0,
 'Нарушение прав акционеров': 0,
 'Обвал': 0,
 'Отзыв продукции': 0,
 'Отмывание денег': 0,
 'Падение метеорита': 0,
 'Падение прибыли': 0,
 'Падение продаж': 0,
 'Поддержка террористов': 0,
 'Пожары': 0,
 'Привлечение к уголовной ответственности': 0,
 'Проблемы с активами': 0,
 'Противодействие коррупции': 0,
 'Растрата денежных средств': 0,
 'Рост убытков': 0,
 'Стихийные бедствия': 0,
 'Теракт': 

The dataset consists of tweets, labeled with one or more emotions.

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [90]:
labels = [label for label in dataset['train'].features if label not in ['ID', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

['O',
 'Авиакатастрофа',
 'Акционерный конфликт',
 'Беспорядки, конфликты и войны',
 'Взрыв газа',
 'Взятки',
 'Возбуждение уголовного дела',
 'Выемки',
 'Жалобы потребителей',
 'Заключение под стражу',
 'Закрытие предприятия или производства',
 'Закрытие филиала, представительства',
 'Землетрясение',
 'Извержение вулкана',
 'Катастрофы, крушения',
 'Кораблекрушение',
 'Коррупция',
 'Манипулирование рынком',
 'Мошенничество',
 'Наводнение',
 'Нападение',
 'Нарушение закона',
 'Нарушение обязанностей',
 'Нарушение прав акционеров',
 'Обвал',
 'Отзыв продукции',
 'Отмывание денег',
 'Падение метеорита',
 'Падение прибыли',
 'Падение продаж',
 'Поддержка террористов',
 'Пожары',
 'Привлечение к уголовной ответственности',
 'Проблемы с активами',
 'Противодействие коррупции',
 'Растрата денежных средств',
 'Рост убытков',
 'Стихийные бедствия',
 'Теракт',
 'Техногенные катастрофы',
 'Убийство',
 'Убытки',
 'Уголовное задержание',
 'Уголовное обвинение',
 'Утечка нефти',
 'Финансовые затруд

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [93]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()

  return encoding

In [94]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [8]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [9]:
tokenizer.decode(example['input_ids'])

"[CLS] “ worry is a down payment on a problem you may never have '. joyce meyer. # motivation # leadership # worry [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [10]:
example['labels']

[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]

In [11]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['anticipation', 'optimism', 'trust']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html).

In [12]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things:

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [14]:
batch_size = 8
metric_name = "f1"

In [15]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [16]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [17]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [18]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
         2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
        14354,  1001,  4105,  1001,  4737,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [19]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


SequenceClassifierOutput(loss=tensor(0.7106, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.7068, -0.3919,  0.6005,  0.2869, -0.3963, -0.1764, -0.0044, -0.3775,
          0.4556, -0.2295, -0.1115]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [20]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-20-c90a37d0d319>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.402300,0.313342,0.692093,0.787808,0.286682
2,0.278800,0.304359,0.693752,0.787140,0.278781
3,0.237900,0.309492,0.709154,0.802956,0.278781
4,0.210100,0.313994,0.706258,0.801636,0.276524
5,0.188400,0.312470,0.707579,0.803025,0.274266


TrainOutput(global_step=4275, training_loss=0.2570351289447985, metrics={'train_runtime': 1008.3227, 'train_samples_per_second': 33.908, 'train_steps_per_second': 4.24, 'total_flos': 2249123476753920.0, 'train_loss': 0.2570351289447985, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [22]:
trainer.evaluate()

{'eval_loss': 0.3094923794269562,
 'eval_f1': 0.7091535433070866,
 'eval_roc_auc': 0.8029556130322487,
 'eval_accuracy': 0.27878103837471785,
 'eval_runtime': 6.7798,
 'eval_samples_per_second': 130.683,
 'eval_steps_per_second': 16.372,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [23]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [24]:
logits = outputs.logits
logits.shape

torch.Size([1, 11])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [25]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['joy', 'optimism']
